In [1]:
project_id = 'pytorch-tpu-nfs'
dataset_id = 'view_dataset'
table_id = 'weather_time_series'
experiment_id = 'weather-exp'
staging_bucket = 'gs://automl-samples'
location='us-central1'
context_window = 24
run_id = f'context-window-{context_window}'

In [4]:
from view_demo.train.custom_tf_trainer import trainer
from view_demo.preprocess.window import Generator as WindowGenerator
from view_demo.models.baseline import linear
experiment_tracking_on = staging_bucket is not None
trainer = trainer(
   project_id,
   location,
   dataset_id,
   table_id
)
trainer.read_dataset()
train_df, val_df, test_df = trainer.create_split()
wide_window = WindowGenerator(
    input_width=context_window, label_width=context_window, shift=1,
    label_columns=['T__degC'],
    train_df=train_df,
    test_df=test_df,
    val_df=val_df)
history = trainer.compile_and_fit(linear, wide_window)
if experiment_tracking_on:
    from google.cloud import aiplatform
    aiplatform.init(
        project=project_id,
        staging_bucket=staging_bucket,
        experiment=experiment_id
    )
    aiplatform.start_run(run=run_id)
    aiplatform.log_metrics({"val_loss": history.history['val_loss'][-1]})
    aiplatform.log_metrics({"val_mae": history.history['val_mean_absolute_error'][-1]})
    aiplatform.log_metrics({"train_loss": history.history['loss'][-1]})
    aiplatform.log_metrics({"train_mae": history.history['mean_absolute_error'][-1]})

10732/10732 [==============================] - 44s 4ms/step - loss: 73.4800 - mean_absolute_error: 3.1547 - val_loss: 3.5103 - val_mean_absolute_error: 0.9004
